# PySpark DataFrame 소개

`config("spark.sql.repl.eagerEval.enabled", True)`  :   

DataFrame을 바로 출력하도록 설정하는 옵션. 기본적으로 PySpark에서는 df.show()를 호출해야 출력되지만, 이 옵션을 True로 설정하면, Jupyter/Colab 등 REPL 환경에서 변수를 실행하면 자동으로 DataFrame 내용이 보여집니다.

In [1]:
from pyspark.sql import SparkSession

# SparkSession은 PySpark 애플리케이션의 진입점으로,
# 데이터프레임 생성, 데이터 읽기/쓰기, SQL 작업 등을 수행할 수 있게 해줍니다.
spark = SparkSession.builder \
    .appName("DataFrame_Practice") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

spark

### 데이터셋 읽기

In [2]:
 # Titanic 데이터셋 CSV 파일 경로
file_location = "/content/titanic.csv"
file_type = "csv"  # 파일 형식을 CSV로 지정

# 지정한 옵션을 사용하여 CSV 파일을 읽어 DataFrame 생성
# schema 자동 추론, 첫 번째 행을 헤더로 간주
df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

df.limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,NULL,S


In [3]:
# 스키마 확인
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
# 데이터프레임(df)이 비어 있는지 여부를 확인
df.isEmpty()

False

In [6]:
# DataFrame의 컬럼 이름 출력
print(df.columns)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [7]:
df.count()

891

#### `Row` 객체 리스트 용도  

- **데이터 미리보기 및 디버깅:**  
  데이터를 처음 불러왔을 때, 상위 몇 개의 행을 확인하여 데이터의 구조나 내용이 예상대로 로드되었는지 검증  

- **개별 행의 데이터 추출:**  
  반환된 `Row` 객체에서 특정 컬럼의 값을 쉽게 추출. 예를 들어, `rows = df.head(3)` 후 `rows[0]['name']` 또는 `rows[0].name`과 같이 사용.  

- **로컬 처리:**  
  소량의 데이터를 로컬 메모리에서 빠르게 처리하거나 확인할 때 유용. (분산 처리 없이 빠른 접근이 필요할 때)

In [9]:
# DataFrame의 상위 3개 행을 Row 객체 리스트로 반환
df.head(3)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C'),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S')]

In [17]:
# DataFrame의 상위 3개 행을 표 형태로 출력
df.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [18]:
# DataFrame의 상위 3개 행을 Pandas DataFrame 형태로 출력
df.toPandas().head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S


In [19]:
# 'Name' 컬럼만 선택하여 상위 5개 행을 가져오기
df.select('Name').limit(5)

Name
"Braund, Mr. Owen ..."
"Cumings, Mrs. Joh..."
"Heikkinen, Miss. ..."
"Futrelle, Mrs. Ja..."
"Allen, Mr. Willia..."


In [20]:
# 'Name'과 'Age' 컬럼을 선택하여 상위 5개 행을 가져오기
df.select(['Name', 'Age']).limit(5)

Name,Age
"Braund, Mr. Owen ...",22.0
"Cumings, Mrs. Joh...",38.0
"Heikkinen, Miss. ...",26.0
"Futrelle, Mrs. Ja...",35.0
"Allen, Mr. Willia...",35.0


In [21]:
# 데이터프레임(df)에서 상위 2개의 행을 가져오는 코드입니다.
# - take(2): 데이터프레임의 첫 번째 2개의 행을 리스트 형식으로 반환.
first_two_rows = df.take(2)

# 가져온 행 데이터를 반복문을 사용하여 출력
# - row: 데이터프레임의 각 행(Row 객체).
for row in first_two_rows:
    print(row)  # 각 행(Row 객체)의 내용을 출력

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S')
Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C')


In [22]:
# DataFrame의 모든 컬럼 이름과 해당 데이터 타입을 튜플의 리스트 형태로 반환
df.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [23]:
# DataFrame의 수치형 컬럼에 대해 기초 통계 정보를 계산
df.describe()

summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891,891,891,891,891,714,891,891,891,891,204,889
mean,446.0,0.3838383838383838,2.308641975308642,NULL,NULL,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,NULL,NULL
stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,NULL,NULL,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,NULL,NULL
min,1,0,1,"""Andersson, Mr. A...",female,0.42,0,0,110152,0.0,A10,C
max,891,1,3,"van Melkebeke, Mr...",male,80.0,8,6,WE/P 5735,512.3292,T,S


- DataFrame.withColumn(colName, col)

PySpark에서 DataFrame에 새로운 컬럼을 추가하거나 기존 컬럼을 업데이트할 때 사용하는 메서드  

| 파라미터  | 설명                                                                 |
|-----------|----------------------------------------------------------------------|
| `colName` | 새로 만들 컬럼의 이름 (문자열)                                       |
| `col`     | 추가하거나 업데이트할 컬럼의 값. 이것은 Expression 또는 Column 객체 여야 함 |


In [27]:
# 'Age' 컬럼의 값이 30보다 큰지(True/False) 여부를 나타내는 새로운 컬럼 'Age Over 30'을 DataFrame에 추가
df = df.withColumn('Age Over 30', df['Age'] > 30)  # 새로운 DataFrame으로 업데이트

df.limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Over 30
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S,false
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C,true
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S,false
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S,true
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,NULL,S,true


In [28]:
# Column 삭제
df = df.drop("Age Over 30")
df.limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,NULL,S


In [29]:
## 컬럼명 변경
df = df.withColumnRenamed('Embarked', 'Port')
df.limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,NULL,S


### Missing Value 처리

In [30]:
from pyspark.sql.functions import col, sum

# 각 컬럼의 null 값 개수 확인
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port
0,0,0,0,0,177,0,0,0,0,687,2


col(c) : 문자열 컬럼 이름 c를 Column 객체로 바꿔줌 (PySpark에서 컬럼을 참조할 때 필요)  
col(c).isNull() : 컬럼의 각 값이 null인지 여부를 Boolean 값으로 반환.  
cast("int") : True/False를 1/0으로 바꿈  
sum(...) : 컬럼에 대해 null인 값(1인 값)의 총합을 계산  
alias(c) : 이 컬럼의 이름을 원래 컬럼 이름으로 설정  
df.select([...]) : 각 컬럼의 null 개수를 계산한 컬럼들만 선택해서 보여줌.

In [35]:
[col(c).isNull().cast("int") for c in df.columns]

[Column<'CAST((PassengerId IS NULL) AS INT)'>,
 Column<'CAST((Survived IS NULL) AS INT)'>,
 Column<'CAST((Pclass IS NULL) AS INT)'>,
 Column<'CAST((Name IS NULL) AS INT)'>,
 Column<'CAST((Sex IS NULL) AS INT)'>,
 Column<'CAST((Age IS NULL) AS INT)'>,
 Column<'CAST((SibSp IS NULL) AS INT)'>,
 Column<'CAST((Parch IS NULL) AS INT)'>,
 Column<'CAST((Ticket IS NULL) AS INT)'>,
 Column<'CAST((Fare IS NULL) AS INT)'>,
 Column<'CAST((Cabin IS NULL) AS INT)'>,
 Column<'CAST((Port IS NULL) AS INT)'>]

Imputer 함수:

| 파라미터      | 설명                                                                 |
|---------------|----------------------------------------------------------------------|
| `inputCols`   | 결측값(null)을 처리할 입력 컬럼 이름 리스트                          |
| `outputCols`  | 처리 결과를 저장할 출력 컬럼 이름 리스트                              |
| `strategy`    | 어떤 방식으로 결측값을 채울지 설정 (`mean`, `median`, `mode` 중 선택) |


In [36]:
from pyspark.ml.feature import Imputer

# Age가 null인 경우 평균으로 채움
imputer = Imputer(
  inputCols = ['Age'],
  outputCols = ['Age_imputed']
).setStrategy('mean')

In [37]:
df = imputer.fit(df).transform(df)
df.limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port,Age_imputed
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S,22.0
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C,38.0
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S,26.0
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S,35.0
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,NULL,S,35.0


## Filtering

- SQL 스타일의 문자열 표현식으로 조건을 지정합니다.
"Age < 30"는 SQL 쿼리의 WHERE 절처럼 동작

In [38]:
## Age 가 30 아래인 사람만 filtering
df.filter("Age < 30").limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port,Age_imputed
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S,22.0
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S,26.0
8,0,3,"Palsson, Master. ...",male,2.0,3,1,349909,21.075,NULL,S,2.0
9,1,3,"Johnson, Mrs. Osc...",female,27.0,0,2,347742,11.1333,NULL,S,27.0
10,1,2,"Nasser, Mrs. Nich...",female,14.0,1,0,237736,30.0708,NULL,C,14.0


- PySpark의 DataFrame API를 사용하여 조건을 지정합니다.
df["Age"] < 30은 PySpark의 컬럼 객체를 사용해 조건을 작성

In [39]:
df.filter(df["Age"] < 30).limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port,Age_imputed
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S,22.0
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S,26.0
8,0,3,"Palsson, Master. ...",male,2.0,3,1,349909,21.075,NULL,S,2.0
9,1,3,"Johnson, Mrs. Osc...",female,27.0,0,2,347742,11.1333,NULL,S,27.0
10,1,2,"Nasser, Mrs. Nich...",female,14.0,1,0,237736,30.0708,NULL,C,14.0


In [40]:
## 특정 column 만 filtering
df.filter("Age < 30").select(["Pclass", "Name", "Age"]).limit(5)

Pclass,Name,Age
3,"Braund, Mr. Owen ...",22.0
3,"Heikkinen, Miss. ...",26.0
3,"Palsson, Master. ...",2.0
3,"Johnson, Mrs. Osc...",27.0
2,"Nasser, Mrs. Nich...",14.0


In [41]:
## AND, OR 조건 추가 - 나이가 20보다 어리거나 60보다 크면서 남성인 사람
df.filter(((df['Age'] < 20) | (df['Age'] > 60)) & (df['Sex'] == 'male')).limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port,Age_imputed
8,0,3,"Palsson, Master. ...",male,2.0,3,1,349909,21.075,NULL,S,2.0
17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.125,NULL,Q,2.0
28,0,1,"Fortune, Mr. Char...",male,19.0,3,2,19950,263.0,C23 C25 C27,S,19.0
34,0,2,"Wheadon, Mr. Edwa...",male,66.0,0,0,C.A. 24579,10.5,NULL,S,66.0
51,0,3,"Panula, Master. J...",male,7.0,4,1,3101295,39.6875,NULL,S,7.0


In [42]:
## Not condition
df.filter(~(df['Age'] < 60)).limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Port,Age_imputed
34,0,2,"Wheadon, Mr. Edwa...",male,66.0,0,0,C.A. 24579,10.5,NULL,S,66.0
55,0,1,"Ostby, Mr. Engelh...",male,65.0,0,1,113509,61.9792,B30,C,65.0
97,0,1,"Goldschmidt, Mr. ...",male,71.0,0,0,PC 17754,34.6542,A5,C,71.0
117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,370369,7.75,NULL,Q,70.5
171,0,1,"Van der hoef, Mr....",male,61.0,0,0,111240,33.5,B19,S,61.0


## GroupBy 와 Aggregation Function (집합 함수)

In [43]:
# 'Sex' 컬럼을 기준으로 그룹화 후, 각 그룹에서 모든 수치형 컬럼의 평균 계산
# 특정 column 만 선택
df.groupBy('Sex').mean().select(['Sex', 'avg(Age)'])

Sex,avg(Age)
female,27.915708812260537
male,30.72664459161148


In [45]:
# 특정 컬럼만 선택 후 그룹화
df[['Sex', 'Age']].groupBy('Sex').mean()

Sex,avg(Age)
female,27.915708812260537
male,30.72664459161148


In [46]:
# 성별 최고령자
df[['Sex', 'Age']].groupBy('Sex').max()

Sex,max(Age)
female,63.0
male,80.0


In [47]:
# 출항 항구별 탑승객 수
df.groupBy('Port').count()

Port,count
Q,77
NULL,2
C,168
S,644


- PySpark DataFrame의 여러 컬럼에 대해 집계(aggregation) 연산을 동시에 수행

In [48]:
# agg 함수 이용 (group by 내포)
df.agg({"Age": "mean", "Fare": "max", "SibSp": "max"})

max(SibSp),avg(Age),max(Fare)
8,29.69911764705882,512.3292


### Parquet 데이터셋 읽기
Parquet는 Apache Hadoop에서 널리 사용되는 컬럼 기반 저장 포맷으로, 대규모 데이터를 효율적으로 저장하고 분석하기 위해 설계되었습니다.

In [49]:
# Parquet 파일 경로 지정
# Python 관련 질문과 답변을 포함한 작은 parqeut 파일
file_path = "/content/0000.parquet"

# Parquet 파일 로드
df = spark.read.parquet(file_path)

In [51]:
# 데이터 확인
df.limit(5)  # 상위 5개 행 출력

output,text,input,instruction
```python\ntasks ...,Help me set up my...,Setting up your d...,Help me set up my...
```python\nshoppi...,Create a shopping...,Creating a shoppi...,Create a shopping...
```python\ntotal_...,Calculate how muc...,Calculating weekl...,Calculate how muc...
```python\ntotal_...,Help me split the...,Splitting the bil...,Help me split the...
```python\nmovie_...,Organize my movie...,Organizing your m...,Organize my movie...


In [52]:
df.printSchema()  # 데이터 스키마 출력

root
 |-- output: string (nullable = true)
 |-- text: string (nullable = true)
 |-- input: string (nullable = true)
 |-- instruction: string (nullable = true)



In [55]:
# 첫 번째 행 가져오기
first_row = df.first()

# 각 컬럼의 값 출력
print("instruction:", first_row.instruction)
print("input:", first_row.input)
print("output:", first_row.output)

instruction: Help me set up my daily to-do list!
input: Setting up your daily to-do list...
output: ```python
tasks = []
while True:
    task = input('Enter a task or type 'done' to finish: ')
    if task == 'done': break
    tasks.append(task)
print(f'Your to-do list for today: {tasks}')
```


## JOIN
기능: 두 데이터프레임 간에 공통 열(키)을 기준으로 행을 결합  
조건: 반드시 두 데이터프레임에 공통 열이 있어야 하며, 이를 기준으로 매칭을 수행  

형태:  

    - INNER JOIN (기본값): 공통 열의 값이 매칭되는 행만 반환.  
    - LEFT JOIN: 왼쪽 데이터프레임의 모든 행과, 매칭되는 오른쪽 데이터프레임의 행을 반환. 매칭되지 않으면 오른쪽 값은 null로 표시.  
    - RIGHT JOIN: 오른쪽 데이터프레임의 모든 행과, 매칭되는 왼쪽 데이터프레임의 행을 반환.  
    - FULL JOIN: 양쪽 데이터프레임의 모든 행을 포함하며, 매칭되지 않는 값은 null로 표시.

<img src="https://i.imgur.com/MfGEduu.png"  width=300 />

In [56]:
# 첫 번째 데이터셋 생성: 이름(Name), 나이(Age), 결혼 상태(Marital Status)를 포함한 데이터
data1 = [
    ("Alice", 34, "Single"),
    ("Bob", 28, "Married"),
    ("Charlie", 31, "Divorced"),
    ("Ronald", 51, "Divorced"),
]

# 첫 번째 데이터셋의 열 이름 정의
columns1 = ["Name", "Age", "Marital Status"]

# PySpark 데이터프레임 생성 (df1)
df1 = spark.createDataFrame(data1, columns1)

# 두 번째 데이터셋 생성: 이름(Name)과 도시(City)를 포함한 데이터
data2 = [
    ("Alice",  "New York"),
    ("Bob", "Los Angeles"),
    ("Charlie", "Chigaco"),
    ("Trump", "San Francisco")
]

# 두 번째 데이터셋의 열 이름 정의
columns2 = ["Name", "City"]

# PySpark 데이터프레임 생성 (df2)
df2 = spark.createDataFrame(data2, columns2)

# 두 데이터프레임(df1, df2)을 공통 열(Name)을 기준으로 내부 조인(inner join) 수행
joined_df = df1.join(df2, on="Name", how="inner")
joined_df

Name,Age,Marital Status,City
Alice,34,Single,New York
Bob,28,Married,Los Angeles
Charlie,31,Divorced,Chigaco


In [57]:
joined_df = df1.join(df2, on="Name", how="left")
joined_df

Name,Age,Marital Status,City
Bob,28,Married,Los Angeles
Alice,34,Single,New York
Ronald,51,Divorced,NULL
Charlie,31,Divorced,Chigaco


In [58]:
joined_df = df1.join(df2, on="Name", how="right")
joined_df

Name,Age,Marital Status,City
Bob,28,Married,Los Angeles
Alice,34,Single,New York
Trump,NULL,NULL,San Francisco
Charlie,31,Divorced,Chigaco


In [59]:
joined_df = df1.join(df2, on="Name", how="full")
joined_df

Name,Age,Marital Status,City
Alice,34,Single,New York
Bob,28,Married,Los Angeles
Charlie,31,Divorced,Chigaco
Ronald,51,Divorced,NULL
Trump,NULL,NULL,San Francisco
